<a href="https://colab.research.google.com/github/Taichi2005/GoogleColab_Whisper/blob/main/%E9%AB%98%E6%80%A7%E8%83%BD%E6%96%87%E5%AD%97%E8%B5%B7%E3%81%93%E3%81%97%E5%AE%9F%E8%A1%8C%E3%82%BB%E3%83%AB_(%E5%8B%95%E7%94%BB%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E8%87%AA%E5%8B%95%E5%AF%BE%E5%BF%9C%E7%89%88).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# セル1: 環境構築
# -------------------------------------------------------------------------------------------
# GPUが有効になっているかを確認
print("▼ GPUの確認")
!nvidia-smi

# faster-whisperと、GPUで動作させるために必要なCUDAライブラリをインストール
print("\n▼ 必要なライブラリのインストール")
!pip install faster-whisper==1.0.3 -q
!pip install nvidia-cublas-cu12==12.6.4.1 -q
!pip install nvidia-cudnn-cu12==9.10.2.21 -q

print("\n✅ 環境構築が完了しました。")

▼ GPUの確認
Thu Oct 23 18:42:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   66C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+--------------------------------------

In [ ]:
# -------------------------------------------------------------------------------------------
# セル2: Google Driveへの接続
# -------------------------------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')
print("\n✅ Google Driveへの接続が完了しました。")

Mounted at /content/drive

✅ Google Driveへの接続が完了しました。


In [ ]:
#@title 🚀 高性能文字起こし実行セル (動画ファイル自動対応版)
#@markdown ### 1. Google Driveのパス設定
#@markdown ---
#@markdown 動画・音声ファイルが保存されているフォルダと、結果を保存するフォルダのパスを指定します。
drive_audio_input_dir = "/content/drive/MyDrive/Colab/Whisper_Transcripts/input_audio" #@param {type:"string"}
drive_transcript_output_dir = "/content/drive/MyDrive/Colab/Whisper_Transcripts/output_transcripts" #@param {type:"string"}

#@markdown ### 2. モデルとパフォーマンス設定
#@markdown ---
#@markdown **モデル**: `deepdml/faster-whisper-large-v3-turbo-ct2`は、`large-v3-turbo`をfaster-whisper用に変換した高速モデルです。<br>
#@markdown **計算タイプ**: `float16`は、精度をほぼ維持したままVRAM使用量を半減させ、高速化する推奨設定です。
model_name = "deepdml/faster-whisper-large-v3-turbo-ct2" #@param ["deepdml/faster-whisper-large-v3-turbo-ct2", "large-v3", "large-v2", "distil-large-v3", "medium"]
compute_type = "float16" #@param ["float16", "int8"]

#@markdown ### 3. VAD (音声区間検出) 設定
#@markdown ---
#@markdown VADを有効にすると、無音区間を自動で除去し、文字起こしの精度を向上させることができます。
use_vad_filter = True #@param {type:"boolean"}
#@markdown ここで指定したミリ秒以上の無音を「発話の区切り」とみなします。
vad_min_silence_duration_ms = 500 #@param {type:"slider", min:100, max:2000, step:100}

# --- ここから下は実行コード（編集不要） ---
#【変更点】必要なライブラリを追加
import os
import glob
import shutil
from faster_whisper import WhisperModel
import time
import subprocess
from datetime import datetime, timezone, timedelta
from tqdm.notebook import tqdm

#【変更点】ヘルパー関数を定義
def get_current_timestamp():
    """現在時刻を[YYYY-MM-DD HH:MM:SS]形式の文字列で返す"""
    JST = timezone(timedelta(hours=+9))
    return datetime.now(JST).strftime('%Y-%m-%d %H:%M:%S')

def preprocess_media_file(drive_path, temp_dir):
    """
    メディアファイル（動画・音声）を前処理する関数。
    動画の場合はFFmpegで音声を抽出し、一時音声ファイルのパスを返す。
    音声の場合はそのままローカルにコピーし、そのパスを返す。
    """
    filename = os.path.basename(drive_path)
    base_filename, file_extension = os.path.splitext(filename)

    local_media_path = os.path.join(temp_dir, filename)
    shutil.copy(drive_path, local_media_path)

    supported_video_formats = ['.mp4', '.mov', '.avi', '.wmv', '.mkv', '.flv', '.webm']
    if file_extension.lower() in supported_video_formats:
        print(f"  - 動画ファイルを検出。音声の抽出を開始...")
        # 抽出後の音声ファイルパスを定義（重複を避けるためタイムスタンプを追加）
        temp_audio_file = os.path.join(temp_dir, f"extracted_audio_{base_filename}_{int(time.time()*1000)}.wav")
        try:
            # ffmpegコマンドで16kHzのモノラル音声(wav)を抽出
            subprocess.run(
                ['ffmpeg', '-i', local_media_path, '-vn', '-ar', '16000', '-ac', '1', '-y', '-loglevel', 'error', temp_audio_file],
                check=True
            )
            os.remove(local_media_path) # 元の動画コピーは不要なので削除
            print(f"  - 音声の抽出が完了 -> {os.path.basename(temp_audio_file)}")
            return temp_audio_file # 抽出した音声ファイルのパスを返す
        except subprocess.CalledProcessError as e:
            print(f"  - 💥 FFmpegエラー ({filename}): 音声の抽出に失敗しました。 {e}")
            if os.path.exists(local_media_path): os.remove(local_media_path)
            return None
    else:
        # 音声ファイルの場合は、コピーしたファイルのパスをそのまま返す
        return local_media_path

# 1. 環境設定とディレクトリの準備
print(f"{get_current_timestamp()} --- 1. 環境設定とディレクトリの準備 ---")
os.makedirs(drive_audio_input_dir, exist_ok=True)
os.makedirs(drive_transcript_output_dir, exist_ok=True)
print(f"入力フォルダ: {drive_audio_input_dir}")
print(f"出力フォルダ: {drive_transcript_output_dir}")

# 2. モデルのロード
print(f"\n{get_current_timestamp()} --- 2. モデル '{model_name}' ({compute_type}) をロード中... ---")
start_load_time = time.time()
model = WhisperModel(model_name, device="cuda", compute_type=compute_type)
end_load_time = time.time()
print(f"✅ モデルのロードが完了しました。({end_load_time - start_load_time:.2f}秒)")

# 3. 処理対象ファイルの検索
print(f"\n{get_current_timestamp()} --- 3. 処理対象ファイルの検索 ---")
supported_formats = ['*.mp3', '*.wav', '*.m4a', '*.flac', '*.ogg', '*.opus', # 音声
                     '*.mp4', '*.mov', '*.avi', '*.wmv', '*.mkv', '*.flv', '*.webm'] # 動画
media_files = []
for fmt in supported_formats:
    media_files.extend(glob.glob(os.path.join(drive_audio_input_dir, fmt)))

if not media_files:
    print("⚠️ 入力フォルダに処理対象のメディアファイルが見つかりませんでした。")
else:
    print(f"✅ {len(media_files)} 件のメディアファイルを検出しました。")
    # 4. メイン処理ループ
    print(f"\n{get_current_timestamp()} --- 4. 文字起こし処理開始 ---")
    #【変更点】tqdmを使用してプログレスバーを表示
    for media_path_drive in tqdm(media_files, desc="全体進捗"):
        start_process_time = time.time()
        filename = os.path.basename(media_path_drive)
        print(f"\n{get_current_timestamp()} ■ 処理開始: {filename}")

        local_audio_path = None # finallyブロックで参照できるよう初期化
        try:
            #【変更点】動画・音声を前処理する関数を呼び出す
            local_audio_path = preprocess_media_file(media_path_drive, temp_dir='/content/')

            # 音声抽出に失敗した場合はスキップ
            if local_audio_path is None:
                print(f"  - ファイル処理をスキップします。")
                continue

            # 文字起こし実行
            print(f"  - 文字起こしを実行中... (VAD: {'有効' if use_vad_filter else '無効'})")
            segments, info = model.transcribe(
                local_audio_path,
                beam_size=5,
                vad_filter=use_vad_filter,
                vad_parameters=dict(min_silence_duration_ms=vad_min_silence_duration_ms)
            )
            print(f"  - 検出言語: {info.language} (確率: {info.language_probability:.2f})")

            # 結果をタイムスタンプ付きテキストとして保存
            base_filename, _ = os.path.splitext(filename) # 出力ファイル名は元のファイル名基準
            output_txt_filename = f"{base_filename}.txt"
            local_output_path = os.path.join('/content/', output_txt_filename)
            drive_output_path = os.path.join(drive_transcript_output_dir, output_txt_filename)

            with open(local_output_path, "w", encoding="utf-8") as f:
                for segment in segments:
                    line = f"[{segment.start:0>7.2f}s -> {segment.end:0>7.2f}s] {segment.text.strip()}\n"
                    f.write(line)

            shutil.move(local_output_path, drive_output_path)
            print(f"  - 結果を保存しました: {drive_output_path}")

        except Exception as e:
            print(f"  - 💥 不明なエラーが発生しました: {e}")

        finally:
            # ローカルの一時ファイルをクリーンアップ
            if local_audio_path and os.path.exists(local_audio_path):
                os.remove(local_audio_path)
            end_process_time = time.time()
            print(f"{get_current_timestamp()} ■ 処理完了 ({end_process_time - start_process_time:.2f}秒)")

    print(f"\n{get_current_timestamp()} --- 🎉 全てのファイルの処理が完了しました ---")

2025-10-24 03:43:03 --- 1. 環境設定とディレクトリの準備 ---
入力フォルダ: /content/drive/MyDrive/Colab/Whisper_Transcripts/input_audio
出力フォルダ: /content/drive/MyDrive/Colab/Whisper_Transcripts/output_transcripts

2025-10-24 03:43:04 --- 2. モデル 'deepdml/faster-whisper-large-v3-turbo-ct2' (float16) をロード中... ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

vocabulary.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

✅ モデルのロードが完了しました。(30.39秒)

2025-10-24 03:43:34 --- 3. 処理対象ファイルの検索 ---
✅ 3 件のメディアファイルを検出しました。

2025-10-24 03:43:34 --- 4. 文字起こし処理開始 ---


全体進捗:   0%|          | 0/3 [00:00<?, ?it/s]


2025-10-24 03:43:34 ■ 処理開始: 2025-10-23 12_57_43.mp3
  - 文字起こしを実行中... (VAD: 有効)
  - 検出言語: ja (確率: 1.00)
  - 結果を保存しました: /content/drive/MyDrive/Colab/Whisper_Transcripts/output_transcripts/2025-10-23 12_57_43.txt
2025-10-24 03:48:29 ■ 処理完了 (295.13秒)

2025-10-24 03:48:29 ■ 処理開始: 2025-10-23 08_41_24.mp3
  - 文字起こしを実行中... (VAD: 有効)
  - 検出言語: ja (確率: 1.00)
  - 結果を保存しました: /content/drive/MyDrive/Colab/Whisper_Transcripts/output_transcripts/2025-10-23 08_41_24.txt
2025-10-24 03:53:57 ■ 処理完了 (327.17秒)

2025-10-24 03:53:57 ■ 処理開始: 2025-10-21 14_45_27.mp3
  - 文字起こしを実行中... (VAD: 有効)
  - 💥 不明なエラーが発生しました: max() iterable argument is empty
2025-10-24 03:54:51 ■ 処理完了 (53.96秒)

2025-10-24 03:54:51 --- 🎉 全てのファイルの処理が完了しました ---
